In [99]:
import pandas as pd
import numpy as np

In [106]:
idx = pd.TimedeltaIndex(np.arange(5) + 1, unit="d", name='time')

ts = pd.DataFrame([idx,[100, 110, 108, 115, 120]]).T.reindex()
ts.columns = ['time', 'value']


In [107]:
ts

,time,value
0,1 days,100
1,2 days,110
2,3 days,108
3,4 days,115
4,5 days,120


In [134]:
X = ts['value'].shift(1)
y = ts['value'].shift(-1)
# y = ts['value'].rolling(1).mean()

In [141]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols = list()

    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))

    for i in range(0, n_out):
        cols.append(df.shift(-i))

    agg = pd.concat(cols, axis=1)
    if dropnan:
      agg.dropna(inplace=True)

    return agg.values


In [145]:
tsa = series_to_supervised(ts, dropnan=False)

In [146]:
pd.DataFrame(tsa)

,0,1,2,3
0,NaT,NaN,1 days,100
1,1 days,100,2 days,110
2,2 days,110,3 days,108
3,3 days,108,4 days,115
4,4 days,115,5 days,120
